In [5]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt

In [6]:
df = pd.read_csv("Book1.csv")
df.head()

,S.N.,Name,Number,Engine Power,Type,Man. Date,Purchased date,Rate (%),Coefficient(k/m),KM
0,110,Toyota Hiace,Ba 1 Pa 1340,2986 CC,Diesel,2006,063/5/9,33.1,0.93,"2,37,679"
1,280,Toyota Hiace,Ba 1 Pa 712,2986 CC,Diesel,2008,066/5/11,26.2,0.71,"3,65,648"
2,82,Toyota Hiace,Ba 1 Pa 771,2986 CC,Diesel,2009,066/12/3,98.6,9.91,NaN
3,134,Toyota Hiace,Ba 1 Pa 766,2986 CC,Diesel,2009,066/12/3,47.4,NaN,"2,62,389"
4,136,Toyota Hiace,Ba 1 Pa 573,2986 CC,Diesel,2009,066/3/11,80.6,3.80,"2,85,000"


In [7]:
df['KM'] = df['KM'].apply(lambda x: pd.to_numeric(str(x).replace(',',''), errors='coerce'))
df = df[df['Rate (%)'].notna()]

In [8]:
df = df[df['Engine Power'].str.contains("CC|cc")]
df['Engine Power'] = pd.to_numeric(df['Engine Power'].str.extract('(\d+)', expand=False))

In [9]:
df_km = df[df['KM'].notna()].drop(["Name", "Number", "Type", "Purchased date", "Coefficient(k/m)"], axis = 1).reset_index(drop=True)
df_km["Man. Date"] = 2020 - df_km["Man. Date"] 

In [10]:
X = df_km[["KM", "Man. Date"]].to_numpy()
y = df_km[["Rate (%)"]].to_numpy()

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 5)
regressor = linear_model.LinearRegression()
regressor.fit(X_train, y_train)

LinearRegression()

In [12]:
predictions = regressor.predict(X_test)

for x in range(len(predictions)):
    print(predictions[x], y_test[x])

[68.44052862] [35.7]
[52.93973157] [47.4]
[71.18069588] [56.4]


In [13]:
model = Sequential()
model.add(Dense(32, input_shape=(2,), activation='tanh'))
# model.add(Dense(16, activation='tanh'))
model.add(Dense(16, activation='relu'))
model.add(Dense(4, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# Compile model
model.compile(loss='mean_squared_error', optimizer='adam')

2022-11-17 05:28:30.166341: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-11-17 05:28:30.171179: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-17 05:28:30.171287: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (shasank-Inspiron-3543): /proc/driver/nvidia/version does not exist
2022-11-17 05:28:30.189065: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [14]:
history = model.fit(X, y, epochs=10000, batch_size = 64, validation_split=0.1)

Epoch 1/10000


In [ ]:
history_df = pd.DataFrame(history.history)
plt.title('model loss')
plt.ylabel('MSE')
plt.xlabel('epoch')
plt.plot(history_df['loss'], label='loss')
plt.plot(history_df['val_loss'], label='val_loss')

plt.legend()